In [16]:
from UtilDatasetPrepFromLORE import *
from Rankers import *
from PreferentialSampling import *
from UniformSampling import *
from StandardEval import *
from FairnessEval import *
from Explainers import *

# ADULT

In [17]:
path_res = '/Users/Marta/FAccT/Res-GERMAN/'
path_data = 'datasets/'
filename = 'adult.csv'
sensitive = ['race'] #OR 'sex','marital-status'

In [7]:
df = pd.read_csv(path_data+filename)#("./doc/example_notebooks/propublica_data_for_fairml.csv")
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [18]:
df = prepare_adult_dataset(filename, path_data, sensitive)
df

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,target,age,workclass,education,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,<=50K,39,6,9,4,0,1,1,2174,0,40,38,0,0,0,0,1
1,<=50K,50,5,9,2,3,0,1,0,0,13,38,0,0,0,0,1
2,<=50K,38,3,11,0,5,1,1,0,0,40,38,0,0,0,0,1
3,<=50K,53,3,1,2,5,0,1,0,0,40,38,0,0,1,0,0
4,<=50K,28,3,9,2,9,5,0,0,0,40,4,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,<=50K,27,3,7,2,12,5,0,0,0,38,38,0,0,0,0,1
32557,>50K,40,3,11,2,6,0,1,0,0,40,38,0,0,0,0,1
32558,<=50K,58,3,11,6,0,4,0,0,0,40,38,0,0,0,0,1
32559,<=50K,22,3,11,4,0,3,1,0,0,20,38,0,0,0,0,1


In [ ]:
'''target = 'target'
column_names = df.columns.values.tolist()
column_names.remove(target)
class_names=['Medium-Low','High']
adClass=0
disClass=1
protected = ['race_African-American','race_Asian','race_Caucasian','race_Hispanic','race_Native-American','race_Other']
protected_feature='races' 
non_sensitive = [elem for elem in column_names if elem not in protected]'''